In [1]:
import numpy as np
np.random.seed(0)

In [2]:
A = np.array([
    [1, 1, 1, 1],
    [1, 1, 0, 0],
    [1, 0, 1, 1],
    [1, 0, 1, 1]
])

In [8]:
X = np.random.uniform(-1, 1, (4, 4))
print(X.shape)
print(X)

(4, 4)
[[ 0.64198646 -0.80579745  0.67588981 -0.80780318]
 [ 0.95291893 -0.0626976   0.95352218  0.20969104]
 [ 0.47852716 -0.92162442 -0.43438607 -0.75960688]
 [-0.4077196  -0.76254456 -0.36403364 -0.17147401]]


In [ ]:
# 가중치 행렬 W의 차원은 (hidden 차원의 수, 노드의 수)
W = np.random.uniform(-1, 1, (2, 4))
print(W.shape)
print(W)

(2, 4)
[[-0.87170501  0.38494424  0.13320291 -0.46922102]
 [ 0.04649611 -0.81211898  0.15189299  0.8585924 ]]


In [10]:
# 어텐션 가중치 행렬은 히든 벡터의 concat에 적용되기 때문에 (1, hidden 차원수 * 2)
W_att = np.random.uniform(-1, 1, (1, 4))
print(W_att.shape)
print(W_att)

(1, 4)
[[-0.3628621   0.33482076 -0.73640428  0.43265441]]


In [ ]:
# 소스 노드와 이웃 노드의 hidden 벡터를 concat
# 두 이웃 노드의 쌍을 얻는 간단한 방법은 COO 형식의 인접 행렬
# COO 형식에서 각 행은 소스 노드와 이웃 노드
connections = np.where(A > 0)
connections

(array([0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3], dtype=int64),
 array([0, 1, 2, 3, 0, 1, 0, 2, 3, 0, 2, 3], dtype=int64))

In [13]:
# np.concatenate를 사용하여 소스 노드와 이웃 노드들의 hidden vector들을 concat
# 즉 connection[0]에 해당하는 소스 노드들에 가중치 행렬 W를 곱한 것과,
# 이웃 노드들을 뜻하는 connections[1]에 가중치 행렬 W를 곱한 값을 concat 
np.concatenate([(X @ W.T)[connections[0]], (X @ W.T)[connections[1]]], axis=1)

array([[-0.40074118,  0.09334253, -0.40074118,  0.09334253],
       [-0.40074118,  0.09334253, -0.8261788 ,  0.4200974 ],
       [-0.40074118,  0.09334253, -0.47334651,  0.05254544],
       [-0.40074118,  0.09334253,  0.09384296,  0.3977991 ],
       [-0.8261788 ,  0.4200974 , -0.40074118,  0.09334253],
       [-0.8261788 ,  0.4200974 , -0.8261788 ,  0.4200974 ],
       [-0.47334651,  0.05254544, -0.40074118,  0.09334253],
       [-0.47334651,  0.05254544, -0.47334651,  0.05254544],
       [-0.47334651,  0.05254544,  0.09384296,  0.3977991 ],
       [ 0.09384296,  0.3977991 , -0.40074118,  0.09334253],
       [ 0.09384296,  0.3977991 , -0.47334651,  0.05254544],
       [ 0.09384296,  0.3977991 ,  0.09384296,  0.3977991 ]])

In [25]:
# 어텐션 행렬 W_att를 곱해주어 선형 변환
a = W_att @ np.concatenate([(X @ W.T)[connections[0]], (X @ W.T)[connections[1]]], axis=1).T
print(a.shape)
print(a)

(1, 12)
[[0.51215937 0.96682539 0.5479752  0.27966998 0.77593887 1.23060489
  0.52484538 0.56066122 0.29235599 0.43463191 0.47044775 0.20214252]]


In [28]:
# a에 활성화 함수를 적용
def leaky_relu(x, alpha=0.2):
    return np.maximum(alpha*x, x)
 
e = leaky_relu(a)
print(e.shape)
print(e)

(1, 12)
[[0.51215937 0.96682539 0.5479752  0.27966998 0.77593887 1.23060489
  0.52484538 0.56066122 0.29235599 0.43463191 0.47044775 0.20214252]]


In [29]:
# 활성화 함수를 통과한 a값을 다시 인접행렬 A와 같은 형태로 복구
# A와 마찬가지로, 연결이 없는 노드쌍은 0으로 나타내고, 연결이 존재하는 노드 쌍에 e 값이 채워짐
# 앞서 정의한 connections에 존재하는 위치 값을 활용
E = np.zeros(A.shape)
E[connections[0], connections[1]] = e[0]
E

array([[0.51215937, 0.96682539, 0.5479752 , 0.27966998],
       [0.77593887, 1.23060489, 0.        , 0.        ],
       [0.52484538, 0.        , 0.56066122, 0.29235599],
       [0.43463191, 0.        , 0.47044775, 0.20214252]])

In [30]:
# attention score에 softmax 정규화를 실시
def softmax2D(x, axis):
    e = np.exp(x - np.expand_dims(np.max(x, axis=axis), axis))
    sum = np.expand_dims(np.sum(e, axis=axis), axis)
    return e / sum
 
W_alpha = softmax2D(E, 1)
print(W_alpha.shape)
print(W_alpha)

(4, 4)
[[0.22703176 0.35772192 0.23531046 0.17993587]
 [0.28602565 0.45067547 0.13164944 0.13164944]
 [0.29234039 0.17296227 0.30300057 0.23169676]
 [0.28764346 0.18624999 0.29813237 0.22797417]]


In [ ]:
# 어텐션 매트릭스는 그래프에 존재하는 모든 노드 pair에 가중치로 적용
# 최종적으로 임베딩 차원이 2인 행렬 H를 각 노드의 표현으로 구함
H = A.T @ W_alpha @ X @ W.T
print(H.shape)
print(H)

(4, 2)
[[-1.78854738  0.95021091]
 [-1.01794311  0.53072614]
 [-1.25162517  0.67489736]
 [-1.25162517  0.67489736]]
